# Command
One or more commands to update the graph's state and send messages to nodes. 四个参数：
- graph：命令要发送到的图，默认当前图
- update：修改图的状态，更新当前图执行中的状态数据。
- resume：恢复中断的执行。用于恢复被 interrupt() 暂停的执行，并告诉图应该如何继续。在 Human-in-the-loop 场景中最常用。
- goto：控制图的执行流程，指定执行完当前节点后，应该去哪个下一个节点。

一些使用案例：
- 组合使用 update + goto：
````py
    def node_a(state: State) -> Command[Literal["node_b", "node_c"]]:
        """根据条件更新状态并路由到不同节点"""
        result = process(state["data"])
        
        # 既要更新状态，又要选择下一个节点
        return Command(
            update={"result": result},  # 更新状态
            goto="node_b" if result > 10 else "node_c"  # 根据结果选择节点
        )
````
- Human-in-the-loop 中使用 resume:
````py
    # 第一次调用：执行到中断点
    result = agent.invoke(
        {"messages": [{"role": "user", "content": "Delete the file"}]},
        config={"configurable": {"thread_id": "thread_1"}}
    )

    # 检查是否中断
    if result.get("__interrupt__"):
        # ... 显示给人工审批 ...
        
        # 第二次调用：用 resume 恢复
        result = agent.invoke(
            Command(
                resume={"decisions": [{"type": "approve"}]}  # 人批准了
            ),
            config={"configurable": {"thread_id": "thread_1"}}  # 同一个线程
        )
````